In [1]:
import findspark
findspark.init()
# Create SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# Aşağıdaki ayarları bilgisayarınızın belleğine göre değiştirebilirsiniz
spark = SparkSession.builder \
.master("local[4]") \
.appName("Dataset-Olusturmak") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

# sparkContext'i kısaltmada tut
sc = spark.sparkContext

In [2]:
my_tuple = ((1,0),(1,1),(0,0),(0,1),(1,0),(1,1),(0,0),(0,1),(1,1),(1,1),(0,0))
                    

In [5]:
matris_df = spark.createDataFrame(my_tuple). \
selectExpr("_1 as label","_2 as prediction")


In [6]:
matris_df.show()

+-----+----------+
|label|prediction|
+-----+----------+
|    1|         0|
|    1|         1|
|    0|         0|
|    0|         1|
|    1|         0|
|    1|         1|
|    0|         0|
|    0|         1|
|    1|         1|
|    1|         1|
|    0|         0|
+-----+----------+



In [9]:
#Pivota geçmeden önce gruplanmış haline bakalım
matris_df.groupBy("label","prediction").count().show()


+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|         0|    2|
|    1|         1|    4|
|    0|         1|    2|
|    0|         0|    3|
+-----+----------+-----+



In [10]:
# 0 yukarıda matris
matris_df.groupBy("label"). \
pivot("prediction", [0,1]). \
count(). \
orderBy("label"). \
show()

+-----+---+---+
|label|  0|  1|
+-----+---+---+
|    0|  3|  2|
|    1|  2|  4|
+-----+---+---+



In [13]:
# 1 yukarıda matris
from pyspark.sql.functions import desc
matris_df.groupBy("label"). \
pivot("prediction", [1,0]). \
count(). \
orderBy(matris_df.label.desc()). \
show()

+-----+---+---+
|label|  1|  0|
+-----+---+---+
|    1|  4|  2|
|    0|  2|  3|
+-----+---+---+

